In [4]:
!pip install  pycryptodome



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 24.4 MB/s eta 0:00:00


In [13]:
import numpy as np
from Crypto.Random import get_random_bytes
import random

# Parameters
m = 10
t = 50
n = 2**m
k = n - m * t

def gf2_matrix_invertible(matrix):

    return np.linalg.matrix_rank(matrix) == len(matrix)

def gf2_matrix_inv(matrix):

    size = len(matrix)
    identity = np.eye(size, dtype=np.uint8)
    augmented = np.hstack((matrix, identity))
    for i in range(size):

        if augmented[i, i] == 0:
            for j in range(i+1, size):
                if augmented[j, i] == 1:
                    augmented[[i, j]] = augmented[[j, i]]
                    break
        augmented[i] = (augmented[i] / augmented[i, i]) % 2

        for j in range(size):
            if i != j:
                augmented[j] = (augmented[j] - augmented[i] * augmented[j, i]) % 2
    return augmented[:, size:]

def generate_random_invertible_matrix(size):
    while True:
        matrix = np.random.randint(0, 2, (size, size), dtype=np.uint8)
        if gf2_matrix_invertible(matrix):
            return matrix

# Key Generation
def key_generation():

    g = [random.randint(1, 2**m - 1) for _ in range(t)]
    alpha = [i for i in range(1, n+1)]
    G = np.random.randint(0, 2, (k, n), dtype=np.uint8)
    S = generate_random_invertible_matrix(k)
    P = np.eye(n, dtype=np.uint8)[np.random.permutation(n)]

    # Compute public key G_hat = S * G * P
    G_hat = (S @ G @ P) % 2

    public_key = G_hat
    private_key = (S, G, P, g, alpha)
    return public_key, private_key

# Encryption
def encrypt(public_key, message):
    if len(message) != k:
        raise ValueError(f"Message length must be {k}")

    G_hat = public_key
    m = np.array([int(bit) for bit in message], dtype=np.uint8)
    c = (m @ G_hat) % 2
    e = np.zeros_like(c)
    error_positions = random.sample(range(len(e)), t)
    for pos in error_positions:
        e[pos] = 1
    c = (c + e) % 2
    return c

# Decryption (using simplified decoding, not Patterson's algorithm)
def decrypt(private_key, ciphertext):
    S, G, P, g, alpha = private_key
    c = np.array(ciphertext, dtype=np.uint8)

    # Simplified: Assuming decoding returns original message without error correction
    P_inv = np.linalg.inv(P).astype(np.uint8) % 2
    S_inv = gf2_matrix_inv(S)
    c_prime = (c @ P_inv) % 2
    m_hat = (c_prime[:k] @ S_inv) % 2  # Ensure we are only using the first k bits
    return ''.join(str(int(bit)) for bit in m_hat)

# Example Usage
public_key, private_key = key_generation()
message = ''.join(random.choice('01') for _ in range(k))  # Example binary message of length k
ciphertext = encrypt(public_key, message)
decrypted_message = decrypt(private_key, ciphertext)

print(f"Original Message: {message}")
print(f"Ciphertext: {''.join(map(str, ciphertext))}")
print(f"Decrypted Message: {decrypted_message}")


Original Message: 01000100011011100100101010101011000101110001010010001000111111111101101000100000000011000001100001111001111011011000101000110110010000110000110110111111011010001000101110101111101101010011111111011001101011000001111101010101011100100010011101011111010111011100011010001001011110110110111001010110010101101111110101000101010000110001111001001011111001010000011111101001111001111000101111100001100101000010011110010101011000100111001110101100011101100110010101110010111001000001100100100000000110000000011011110101100000011010
Ciphertext: 1001010011011101010001101100111001000100100110010100111110010011110000111000000011000110110011110010001001111010001100010110010010010101000110000001101101001001001011100010111100100111100011010101010000010110011100001110011011110100100101011111001101110111100101100111000110100001000101110100001010001000110000101010011111010100101111001000110101101010010110010111111010010100000110011100001100010001101000001100110101111010110100010110111001011

<ipython-input-13-3a82fffc673f>:27: RuntimeWarning: divide by zero encountered in divide
  augmented[i] = (augmented[i] / augmented[i, i]) % 2
<ipython-input-13-3a82fffc673f>:27: RuntimeWarning: invalid value encountered in divide
  augmented[i] = (augmented[i] / augmented[i, i]) % 2
<ipython-input-13-3a82fffc673f>:27: RuntimeWarning: invalid value encountered in remainder
  augmented[i] = (augmented[i] / augmented[i, i]) % 2
<ipython-input-13-3a82fffc673f>:27: RuntimeWarning: invalid value encountered in cast
  augmented[i] = (augmented[i] / augmented[i, i]) % 2


In [12]:
def test_key_generation():
    public_key, private_key = key_generation()
    assert public_key.shape == (k, n), "Public key shape mismatch"
    assert len(private_key) == 5, "Private key should have 5 components"
    print("Key generation test passed.")

def test_encryption_decryption():
    public_key, private_key = key_generation()
    message = ''.join(random.choice('01') for _ in range(k))  # Example binary message of length k
    ciphertext = encrypt(public_key, message)
    decrypted_message = decrypt(private_key, ciphertext)
    assert message == decrypted_message, "Decryption failed"
    print("Encryption/Decryption test passed.")

def test_incorrect_message_length():
    public_key, private_key = key_generation()
    try:
        message = '101010'  # Incorrect length
        ciphertext = encrypt(public_key, message)
        decrypted_message = decrypt(private_key, ciphertext)
        assert False, "Encryption should fail with incorrect message length"
    except ValueError as e:
        print(f"Expected error for incorrect message length: {e}")

def test_random_message_encryption_decryption():
    public_key, private_key = key_generation()
    message = ''.join(random.choice('01') for _ in range(k))
    ciphertext = encrypt(public_key, message)
    decrypted_message = decrypt(private_key, ciphertext)
    assert message == decrypted_message, "Decryption failed for random message"
    print("Random message encryption/decryption test passed.")

def test_resilience_to_error_in_ciphertext():
    public_key, private_key = key_generation()
    message = ''.join(random.choice('01') for _ in range(k))
    ciphertext = encrypt(public_key, message)

    # Introduce additional errors to ciphertext
    error_positions = random.sample(range(n), t // 2)  # Introducing t/2 errors
    for pos in error_positions:
        ciphertext[pos] ^= 1  # Flip the bit

    decrypted_message = decrypt(private_key, ciphertext)
    # Note: Simplified version won't handle additional errors correctly
    print(f"Original Message: {message}")
    print(f"Decrypted Message with additional errors: {decrypted_message}")
    print("Note: In a real system, decryption might fail or partially recover the message.")

def test_empty_message():
    public_key, private_key = key_generation()
    try:
        message = ''  # Empty message
        ciphertext = encrypt(public_key, message)
        decrypted_message = decrypt(private_key, ciphertext)
        assert False, "Encryption should fail with empty message"
    except ValueError as e:
        print(f"Expected error for empty message: {e}")

# Run the test cases
test_key_generation()
test_encryption_decryption()
test_incorrect_message_length()
test_random_message_encryption_decryption()
test_resilience_to_error_in_ciphertext()
test_empty_message()


Key generation test passed.


AssertionError: Decryption failed